In [9]:
# installs

In [10]:
import gradio as gr
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import json
import matplotlib.pyplot as plt
import io
import os

# ============================================================================
# STATELESS HELPER CLASSES - IMPLEMENT THE CORE FUNCTIONS
# ============================================================================

class Calib:
    @staticmethod
    def load_K(json_data):
        data = json.loads(json_data) if isinstance(json_data, str) else json_data
        return np.array(data.get('K', np.eye(3)), dtype=np.float32), np.array(data.get('distCoeffs', [0]*5), dtype=np.float32)

class Model:
    @staticmethod
    def load_points(data):
        points = json.loads(data) if isinstance(data, str) else data
        points = np.array(points, dtype=np.float32)
        return points if points.shape[1] == 3 else np.column_stack([points, np.zeros(len(points))])

class Clicks:
    @staticmethod
    def capture(points_list):
        return np.array(points_list, dtype=np.float32) if points_list else np.array([]).reshape(0, 2)
    
    @staticmethod
    def draw_numbered(image, points, radius=8):
        if isinstance(image, np.ndarray): image = Image.fromarray(image)
        img = image.copy()
        draw = ImageDraw.Draw(img)
        try: font = ImageFont.truetype("arial.ttf", 16)
        except: font = ImageFont.load_default()
        for i, (x, y) in enumerate(points):
            draw.ellipse([x-radius, y-radius, x+radius, y+radius], outline=(255, 0, 0), width=2, fill=(255, 255, 255))
            draw.text((x-5, y-8), str(i+1), fill=(0, 0, 0), font=font)
        return img

class Pose:
    @staticmethod
    def from_homography(K, model_2D, image_2D):
        """IMPLEMENT: Homography->Pose decomposition"""
        return np.eye(3, dtype=np.float32), np.zeros((3, 1), dtype=np.float32)  # Placeholder
    
    @staticmethod
    def from_opencv(K, distCoeffs, model_3D, image_2D):
        """IMPLEMENT: OpenCV solvePnP"""
        return np.eye(3, dtype=np.float32), np.zeros((3, 1), dtype=np.float32)  # Placeholder

class Viz:
    
    @staticmethod
    def overlay_reprojection(image, model_3D, image_2D, K, distCoeffs, R, t):
        """IMPLEMENT: Show reprojection overlay"""
        if isinstance(image, np.ndarray): image = Image.fromarray(image)
        img = image.copy()
        draw = ImageDraw.Draw(img)
        for x, y in image_2D:
            draw.ellipse([x-5, y-5, x+5, y+5], outline=(255, 0, 0), width=2)
        return img
    
    @staticmethod
    def plot_camera_3d(R, t):
        """IMPLEMENT: 3D camera pose visualization"""
        fig = plt.figure(figsize=(6, 5))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter([0], [0], [0], color='black', s=100, label='World Origin')
        cam_pos = -R.T @ t
        ax.scatter(cam_pos[0], cam_pos[1], cam_pos[2], color='red', s=100, label='Camera')
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z'); ax.set_title('Camera Pose'); ax.legend()
        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=100, bbox_inches='tight')
        buf.seek(0)
        img = Image.open(buf)
        plt.close(fig)
        return img




In [11]:
# ============================================================================
# SINGLE-CELL GRADIO DEMO
# ============================================================================

def create_pose_estimation_demo():
    state = {'points': [], 'model_3d': None, 'K': None, 'dist': None, 'image': None}
    
    def on_click(evt: gr.SelectData, img):
        if img is None: return img, ""
        x, y = evt.index
        state['points'].append([int(x), int(y)])
        state['image'] = img
        points_text = "\n".join([f"Point {i+1}: ({x}, {y})" for i, (x, y) in enumerate(state['points'])])
        return np.array(Clicks.draw_numbered(img, state['points'])), points_text
    
    def undo_last(img):
        if state['points']: state['points'].pop()
        points_text = "\n".join([f"Point {i+1}: ({x}, {y})" for i, (x, y) in enumerate(state['points'])])
        return np.array(Clicks.draw_numbered(img, state['points'])) if img is not None and state['points'] else img, points_text

    
    def clear_points(img):
        state['points'].clear()
        return img, ""
    
    def load_intrinsics_auto():
        if os.path.exists("calibration.json"):
            try:
                with open("calibration.json", 'r') as f:
                    data = json.load(f)
                state['K'], state['dist'] = Calib.load_K(data)
                return f"Loaded from calibration.json: K shape {state['K'].shape}, dist shape {state['dist'].shape}"
            except Exception as e:
                return f"Error loading calibration.json: {e}"
        return "No calibration.json found - enter manually"

    def load_intrinsics_text(text):
        if not text.strip(): return "No text entered"
        try:
            state['K'], state['dist'] = Calib.load_K(text)
            return f"Loaded from text: K shape {state['K'].shape}, dist shape {state['dist'].shape}"
        except Exception as e:
            return f"Error parsing text: {e}"
    
    def load_model_points(text):
        try:
            state['model_3d'] = Model.load_points(text)
            return f"Loaded {len(state['model_3d'])} points"
        except:
            return "Error loading points"
    
    def estimate_poses(intrinsics_text):
        if not state['points'] or not intrinsics_text: return "Missing data"
        try:
            state['K'], state['dist'] = Calib.load_K(intrinsics_text)
            image_pts = Clicks.capture(state['points'])
            
            if state['model_3d'] is None or len(image_pts) != len(state['model_3d']):
                return "Point count mismatch"
            
            # Homography method (2D)
            R_h, t_h = Pose.from_homography(state['K'], state['model_3d'][:, :2], image_pts)
            
            # OpenCV method (3D)
            R_cv, t_cv = Pose.from_opencv(state['K'], state['dist'], state['model_3d'], image_pts)
            
            return f"Homography R,t:\n{R_h}\n{t_h.flatten()}\n\nOpenCV R,t:\n{R_cv}\n{t_cv.flatten()}"
        except Exception as e:
            return f"Estimation failed: {e}"
    
    def create_overlay():
        if not all([state['image'], state['points'], state['model_3d'], state['K']]):
            return state['image']
        try:
            image_pts = Clicks.capture(state['points'])
            R, t = Pose.from_opencv(state['K'], state['dist'], state['model_3d'], image_pts)
            overlay = Viz.overlay_reprojection(state['image'], state['model_3d'], image_pts, state['K'], state['dist'], R, t)
            return np.array(overlay)
        except:
            return state['image']
    
    def create_3d_plot():
        if not all([state['points'], state['model_3d'], state['K']]):
            return None
        try:
            image_pts = Clicks.capture(state['points'])
            R, t = Pose.from_opencv(state['K'], state['dist'], state['model_3d'], image_pts)
            return Viz.plot_camera_3d(R, t)
        except:
            return None
    
    with gr.Blocks(title="Pose Estimation") as demo:
        gr.Markdown("# Pose Estimation Interface")
        gr.Markdown("Upload image, set intrinsics, define model points, click features, estimate pose")
        
        with gr.Row():
            with gr.Column():
                img = gr.Image(label="Image", type="numpy", interactive=True)
                with gr.Row():
                    undo_btn = gr.Button("Undo")
                    clear_btn = gr.Button("Clear")
            
            with gr.Column():
                intrinsics_txt = gr.Textbox(label="Camera Intrinsics", lines=3,
                    placeholder='{"K": [[800,0,320],[0,800,240],[0,0,1]], "distCoeffs": [0,0,0,0,0]}')
                intrinsics_status = gr.Textbox(label="Intrinsics Status")
                
                model_points_txt = gr.Textbox(label="Model Points", lines=3,
                    placeholder='[[0,0,0],[1,0,0],[1,1,0],[0,1,0]]')
                model_status = gr.Textbox(label="Model Points Status")
                
                points_coordinates = gr.Textbox(label="Clicked Points", lines=4, interactive=False)
        
        with gr.Row():
            estimate_btn = gr.Button("Estimate Pose", variant="primary")
            overlay_btn = gr.Button("Show Overlay")
        
        with gr.Tabs():
            with gr.Tab("Pose Results"):
                results_txt = gr.Textbox(label="Results", lines=8, interactive=False)
            
            with gr.Tab("3D Visualization"):
                plot_3d = gr.Image(label="Camera Pose")
            
            with gr.Tab("Overlay"):
                overlay_img = gr.Image(label="Reprojection Overlay")
        
        # Event handlers
        img.select(on_click, inputs=[img], outputs=[img, points_coordinates])
        undo_btn.click(undo_last, inputs=[img], outputs=[img, points_coordinates])
        clear_btn.click(clear_points, inputs=[img], outputs=[img, points_coordinates])
        intrinsics_txt.change(load_intrinsics_text, inputs=[intrinsics_txt], outputs=[intrinsics_status])
        model_points_txt.change(load_model_points, inputs=[model_points_txt], outputs=[model_status])
        estimate_btn.click(estimate_poses, inputs=[intrinsics_txt], outputs=[results_txt])
        overlay_btn.click(create_overlay, outputs=[overlay_img])
        plot_3d.change(lambda: create_3d_plot(), outputs=[plot_3d])

        demo.load(load_intrinsics_auto, outputs=[intrinsics_status])
    
    return demo

In [12]:
# Launch the demo
demo = create_pose_estimation_demo()
demo.launch(debug=False, share=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://785b7ae09615045f41.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
